# How-To, First Steps

## Load package and create basic options

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))

from robustfpm.finance import make_option

Let's start with some basic options. Remember that 1D PutOnMax is just Put (the same goes for CallOnMax)

In [2]:
option1 = make_option(option_type='putonmax', strike=90)
option2 = make_option(option_type='putonmax', strike=80, payoff_dates = 5)
option3 = make_option(option_type='callonmax', strike=90, payoff_dates = [3,5])
option4 = make_option(option_type='put2call1') # American Put2Call1 option

Now let's see how these options got instantiated

In [4]:
print('Option 1: {}'.format(option1))
print('Option 2: {}'.format(option2))
print('Option 3: {}'.format(option3))
print('Option 4: {}'.format(option4))

Option 1: <robustfpm.finance.derivatives.AmericanOption object at 0x0000000005BF5D30>
Option 2: <robustfpm.finance.derivatives.EuropeanOption object at 0x0000000005BF5DA0>
Option 3: <robustfpm.finance.derivatives.BermudanOption object at 0x0000000005BF5DD8>
Option 4: <robustfpm.finance.derivatives.AmericanOption object at 0x000000000598E8D0>


In [5]:
print('Option 1 payoff at x = 80, 90, and 100, t = 3: {}'.format(option1.payoff([[80], [90], [100]], 3)))
print('Option 2 payoff at x = 100, t = 3: {}'.format(option2.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 3: {}'.format(option3.payoff(100, 3)))
print('Option 3 payoff at x = 100, t = 4: {}'.format(option3.payoff(100, 4)))

Option 1 payoff at x = 80, 90, and 100, t = 3: [10.  0.  0.]
Option 2 payoff at x = 100, t = 3: [-inf]
Option 3 payoff at x = 100, t = 3: [10.]
Option 3 payoff at x = 100, t = 4: [-inf]


## Creating solver and solving some problems

Now let's create some basic 1D Problem with Rectangular multiplicative dynamics and no trading constraints. For that, we need module `robustfpm.pricing`

In [6]:
from robustfpm.pricing import *
import numpy as np
pm1 = Problem(starting_price=np.array(100), 
            price_dynamics=ConstantDynamics(support=RectangularHandler([.9, 1.1]), type='mult'),
            trading_constraints=NoConstraints, option=option1, 
            lattice=Lattice(delta=[1]), 
            time_horizon=5)

Now we create solver with some parameters.

Most of the time, there is no point in tweaking *all* of these parameters, only some, namely `enable_timer` and `iter_tick`

In [7]:
opts = {'convex_hull_filter': 'qhull', 'convex_hull_prune_fail_count': 0,
        'convex_hull_prune_success_count':0,'convex_hull_prune_corner_n': 3,'convex_hull_prune_seed': 0} 
solver = ConvhullSolver(enable_timer=True, pricer_options=opts, ignore_warnings=True, iter_tick=50)

Now we solve it and see the result $V_0(x_0)$

In [8]:
sol1 = solver.solve(pm1)
# the solution is simply a dictionary
print('Value: {0}'.format(sol1['Vf'][0][0]))

Precalculating points for value function evaluation: 0.1060 sec (CPU 0.1064 sec)
Computing value function in the last point: 0.0000 sec (CPU 0.0000 sec)
t = 4
iter = 4/90 (4.44%)
iter = 22/90 (24.44%)
iter = 23/90 (25.56%)
iter = 24/90 (26.67%)
iter = 46/90 (51.11%)
t = 3
iter = 14/67 (20.90%)
iter = 44/67 (65.67%)
iter = 65/67 (97.01%)
t = 2
iter = 0/45 (0.00%)
iter = 6/45 (13.33%)
t = 1
t = 0
Computing value function in intermediate points in time: 1.0621 sec (CPU 1.0623 sec)
Solving the problem: 1.1701 sec (CPU 1.1696 sec)
Value: 5.327786420207287


Let's play around and change Lattice step

In [9]:
pm1.lattice = Lattice(delta=[.1])
sol2 = solver.solve(pm1)
print('Value: {0}'.format(sol2['Vf'][0][0]))

Precalculating points for value function evaluation: 0.9381 sec (CPU 0.9383 sec)
Computing value function in the last point: 0.0010 sec (CPU 0.0003 sec)
t = 4
iter = 23/818 (2.81%)
iter = 82/818 (10.02%)
iter = 122/818 (14.91%)
iter = 175/818 (21.39%)
iter = 215/818 (26.28%)
iter = 221/818 (27.02%)
iter = 251/818 (30.68%)
iter = 256/818 (31.30%)
iter = 260/818 (31.78%)
iter = 291/818 (35.57%)
iter = 326/818 (39.85%)
iter = 336/818 (41.08%)
iter = 375/818 (45.84%)
iter = 468/818 (57.21%)
iter = 590/818 (72.13%)
iter = 612/818 (74.82%)
iter = 646/818 (78.97%)
iter = 652/818 (79.71%)
iter = 690/818 (84.35%)
iter = 707/818 (86.43%)
t = 3
iter = 82/609 (13.46%)
iter = 89/609 (14.61%)
iter = 116/609 (19.05%)
iter = 122/609 (20.03%)
iter = 127/609 (20.85%)
iter = 175/609 (28.74%)
iter = 193/609 (31.69%)
iter = 197/609 (32.35%)
iter = 225/609 (36.95%)
iter = 334/609 (54.84%)
iter = 353/609 (57.96%)
iter = 441/609 (72.41%)
iter = 475/609 (78.00%)
iter = 477/609 (78.33%)
iter = 487/609 (79.97%)


Let's try 2D Problem with another option and *additive* dynamics.

In [10]:
pm2 = Problem(starting_price=np.array([91,90]), price_dynamics=ConstantDynamics(support=RectangularHandler([[-1, 1],[-.75, 1]]), type='add'),
             trading_constraints=IdenticalMap(RealSpaceHandler()),
             option=option4,
             lattice=Lattice(delta=[.1,.1]), time_horizon=5)
solver.iter_tick = 200
sol3 = solver.solve(pm2)
print('Value: {0}'.format(sol3['Vf'][0][0]))

Precalculating points for value function evaluation: 1.0551 sec (CPU 1.0547 sec)
Computing value function in the last point: 0.0010 sec (CPU 0.0004 sec)
t = 4
iter = 49/5913 (0.83%)
iter = 81/5913 (1.37%)
iter = 267/5913 (4.52%)
iter = 557/5913 (9.42%)
iter = 954/5913 (16.13%)
iter = 1068/5913 (18.06%)
iter = 2374/5913 (40.15%)
iter = 2459/5913 (41.59%)
iter = 2674/5913 (45.22%)
iter = 2687/5913 (45.44%)
iter = 2849/5913 (48.18%)
iter = 3661/5913 (61.91%)
iter = 3750/5913 (63.42%)
iter = 4502/5913 (76.14%)
iter = 4737/5913 (80.11%)
iter = 4941/5913 (83.56%)
iter = 5028/5913 (85.03%)
iter = 5308/5913 (89.77%)
iter = 5513/5913 (93.24%)
iter = 5541/5913 (93.71%)
iter = 5607/5913 (94.82%)
t = 3
iter = 3/3355 (0.09%)
iter = 91/3355 (2.71%)
iter = 417/3355 (12.43%)
iter = 494/3355 (14.72%)
iter = 594/3355 (17.70%)
iter = 627/3355 (18.69%)
iter = 675/3355 (20.12%)
iter = 1195/3355 (35.62%)
iter = 1209/3355 (36.04%)
iter = 1234/3355 (36.78%)
iter = 1347/3355 (40.15%)
iter = 1439/3355 (42.89%)
